In [7]:
# EES10_14_18.ipynb
# Project: OBSERVATORIO LA CAIXA
# Converted from Stata to Python
# 17/02/2021, version 1
# Manuel Alejandro Hidalgo (Universidad Pablo de Olavide)

import pandas as pd
import numpy as np
import os
import patoolib
import shutil

In [8]:


# Define paths relative to the 'src' directory where the notebook is
zip_path = os.path.join('..', 'data', 'EES10_14_18.zip')
extract_path = os.path.join('..', 'data')
# This will be the base path for the loaded data, assuming the zip extracts a folder with its name
data_base_path = os.path.join(extract_path, 'EES10_14_18')

# Extract data
print(f"Extracting {zip_path}...")
if not os.path.exists(data_base_path):
    patoolib.extract_archive(zip_path, outdir=extract_path)
    print(f"Extraction completed in {extract_path}")
else:
    print(f"Data already extracted in {data_base_path}")

Extracting ..\data\EES10_14_18.zip...
Data already extracted in ..\data\EES10_14_18


In [9]:


# Load 2018 data
print("Loading 2018 data...")
df = pd.read_stata(os.path.join(data_base_path, 'datos_salarial18', 'EES18.dta'))
print(f"2018 data loaded: {df.shape[0]} observations")
df['year'] = 2018

# Ajuste nombre y variables a modelo 10_14
df = df.rename(columns={
    'idenccc': 'ordenccc',
    'nuts1': 'region',
    'cnace': 'secc',
    'regulacion': 'convenio',
    'cno1': 'cno2',
    'estu': 'cestudio',  # similar al de 14
    'anos2': 'edad'
})

print("Starting variable transformations...")

# Generate new variables
df['salbruto'] = df['retrinoin'] + df['retriin']
df['vesp'] = df['vespnoin'] + df['vespin']
print("Basic salary variables created.")

# Convert string variables to numeric (equivalent to destring)
df['sexo'] = pd.to_numeric(df['sexo'], errors='coerce')
df['edad'] = pd.to_numeric(df['edad'], errors='coerce')
df['tipojor'] = pd.to_numeric(df['tipojor'], errors='coerce')
df['control'] = pd.to_numeric(df['control'], errors='coerce')
df['mercado'] = pd.to_numeric(df['mercado'], errors='coerce')
df['convenio'] = pd.to_numeric(df['convenio'], errors='coerce')
df['tipopais'] = pd.to_numeric(df['tipopais'], errors='coerce')
print("String to numeric conversions completed.")

# Load 2022 data
print("Loading 2022 data...")
df_2022 = pd.read_stata(os.path.join(data_base_path, 'datos_salarial22', 'EES_2022.dta'))
df_2022.columns = df_2022.columns.str.lower()
print(f"2022 data loaded: {df.shape[0]} observations")
df_2022['year'] = 2022

# Ajuste nombre y variables a modelo 10_14
df_2022 = df_2022.rename(columns={
    'idenccc': 'ordenccc',
    'nuts1': 'region',
    'cnace': 'secc',
    'regulacion': 'convenio',
    'cno1': 'cno2',
    'estu': 'cestudio',  # similar al de 14
    'anos2': 'edad'
})

print("Starting variable transformations...")

# Generate new variables
df_2022['salbruto'] = df_2022['retrinoin'] + df_2022['retriin']
df_2022['vesp'] = df_2022['vespnoin'] + df_2022['vespin']
print("Basic salary variables created.")

# Convert string variables to numeric (equivalent to destring)
df_2022['sexo'] = pd.to_numeric(df_2022['sexo'], errors='coerce')
df_2022['edad'] = pd.to_numeric(df_2022['edad'], errors='coerce')
df_2022['tipojor'] = pd.to_numeric(df_2022['tipojor'], errors='coerce')
df_2022['control'] = pd.to_numeric(df_2022['control'], errors='coerce')
df_2022['mercado'] = pd.to_numeric(df_2022['mercado'], errors='coerce')
df_2022['convenio'] = pd.to_numeric(df_2022['convenio'], errors='coerce')
df_2022['tipopais'] = pd.to_numeric(df_2022['tipopais'], errors='coerce')
print("String to numeric conversions completed.")


df = pd.concat([df, df_2022], ignore_index=True, sort=False)

# Append 2010 data
print("Loading 2010 data...")
df_2010 = pd.read_stata(os.path.join(data_base_path, 'datos_salarial10', 'EES10.dta'))
print(f"2010 data loaded: {df_2010.shape[0]} observations")
df = pd.concat([df, df_2010], ignore_index=True, sort=False)
print(f"Combined dataset after 2010: {df.shape[0]} observations")

# Replace cestudio for 2010 data
df.loc[(df['cestudio'] == "6") & (df['year'] == 2010), 'cestudio'] = "5"
df.loc[(df['cestudio'] == "7") & (df['year'] == 2010), 'cestudio'] = "6"
df.loc[(df['cestudio'] == "8") & (df['year'] == 2010), 'cestudio'] = "7"

# Append 2014 data
df_2014 = pd.read_stata(os.path.join(data_base_path, 'datos_salarial14', 'EES14.dta'))
df = pd.concat([df, df_2014], ignore_index=True, sort=False)

# Replace cestudio
df.loc[df['cestudio'] == "5", 'cestudio'] = "4"

# Generate new variables
print("Creating salary and time variables...")
df['diasrelaba'] = df['drelabam'] * 30.42 + df['drelabad']
df.loc[df['diasrelaba'] > 365, 'diasrelaba'] = 365

df['diasano'] = df['diasrelaba'] - df['dsiespa2'] - df['dsiespa4']

# Salario anual total
df['salbase'] = (365 / df['diasano']) * (df['salbruto'] + df['vesp']) / 12
# Salario mensual total
df['salmes'] = df['salbase'] + df['comsal'] + df['phextra'] + df['extraorm']
# Salario neto
df['salneto'] = df['salmes'] - df['cotiza'] - df['irpfmes']

df['horas'] = ((df['jsp1'] + (df['jsp2'] / 60)) * 4.35) + df['hextra']
df['salhora'] = df['salmes'] / df['horas']
# Sin horas extras
df['salbasehora'] = df['salbase'] / ((df['jsp1'] + (df['jsp2'] / 60)) * 4.35)
df['restosal'] = df['salhora'] - df['salbasehora']

# Precios constantes de 2010
df.loc[df['year'] == 2014, 'salhora'] = df.loc[df['year'] == 2014, 'salhora'] * 96.903 / 103.732

df['lsalhora'] = np.log(df['salhora'])
df['lsalbasehora'] = np.log(df['salbasehora'])
df['lrestosal'] = np.log(df['restosal'])
df['horascontr'] = np.log(df['jsp1'])
print("Salary and hours calculations completed.")

# Recode variables
df.loc[df['sexo'] == 1, 'sexo'] = 0
df.loc[df['sexo'] == 6, 'sexo'] = 1

df['control'] = df['control'] - 1
df['tipopais'] = df['tipopais'] - 1

df['tipojor'] = pd.to_numeric(df['tipojor'], errors='coerce')
df.loc[df['tipojor'] == 1, 'tipojor'] = 0
df.loc[df['tipojor'] == 2, 'tipojor'] = 1
df = df.rename(columns={'tipojor': 'parcial'})

df['tipoconb'] = pd.to_numeric(df['tipocon'], errors='coerce')
df = df.drop(columns=['tipocon'])
df = df.rename(columns={'tipoconb': 'tipocon'})
df.loc[df['tipocon'] == 2, 'tipocon'] = 0
df = df.rename(columns={'tipocon': 'indefin'})

# Create age dummy variables
print("Creating dummy variables...")
edad_dummies = pd.get_dummies(df['edad'], prefix='age')
df = pd.concat([df, edad_dummies], axis=1)

# Create sector dummy variables
sect_dummies = pd.get_dummies(df['secc'], prefix='sect')
df = pd.concat([df, sect_dummies], axis=1)
print("Age and sector dummies created.")

# Label sector variables (as comments since Python doesn't have variable labels like Stata)
# sect1: "industrias extractivas"
# sect2: "alimentación y bebidas"
# sect3: "madera y papel"
# sect4: "artes graficas"
# sect5: "química, farmacia y refino"
# sect6: "otros prod. minerales no met."
# sect7: "metalurgia"
# sect8: "prod. informáticos, electrónicos y ópticos"
# sect9: "fab. mat. de transporte y muebles"
# sect10: "e. eléctrica, gas, vapor"
# sect11: "suministro de agua"
# sect12: "construcción"
# sect13: "comercio al por mayor"
# sect14: "comercio al por menor"
# sect15: "transporte"
# sect16: "almacenamiento"
# sect17: "hostelería"
# sect18: "información y comunicaciones"
# sect19: "act. financieras y de seguros"
# sect20: "act. inmobiliarias"
# sect21: "act. profesionales, científicas y técnicas"
# sect22: "act. administrativas y servicios auxliares"
# sect23: "aapp y defensa"
# sect24: "educación"
# sect25: "act. sanitarias y de servicios sociales"
# sect26: "act. artísticas y recreativas"
# sect27: "otros servicios"

# ICT variable
df['ict'] = 0
# Check if columns exist before using them
sect_cols = [col for col in df.columns if col.startswith('sect')]
if 'sect_8' in df.columns:
    df.loc[(df['sect_8'] == 1) | (df['sect_18'] == 1) | (df['sect_19'] == 1) | 
           (df['sect_21'] == 1) | (df['sect_26'] == 1), 'ict'] = 1

# High Value added
df['v_added'] = 0
if 'sect_18' in df.columns:
    df.loc[(df['sect_18'] == 1) | (df['sect_5'] == 1) | (df['sect_9'] == 1) | 
           (df['sect_19'] == 1) | (df['sect_8'] == 1), 'v_added'] = 1

# Create market dummy variables
market_dummies = pd.get_dummies(df['mercado'], prefix='market')
df = pd.concat([df, market_dummies], axis=1)

# Create convenio dummy variables
conv_dummies = pd.get_dummies(df['convenio'], prefix='conv_')
df = pd.concat([df, conv_dummies], axis=1)

# Create estudios dummy variables
estudios_dummies = pd.get_dummies(df['cestudio'], prefix='estudios')
df = pd.concat([df, estudios_dummies], axis=1)

# Generate anoanti2
df['anoanti2'] = df['anoanti'] ** 2

# Generate contrato variable
df['contrato'] = 0
df.loc[(df['indefin'] == 1) & (df['parcial'] == 0), 'contrato'] = 1
df.loc[(df['indefin'] == 1) & (df['parcial'] == 1), 'contrato'] = 2
df.loc[(df['indefin'] == 0) & (df['parcial'] == 0), 'contrato'] = 3
df.loc[(df['indefin'] == 0) & (df['parcial'] == 1), 'contrato'] = 4

print("Data processing completed successfully!")
print(f"Final dataset shape: {df.shape}")
print(f"Years in dataset: {sorted(df['year'].unique())}")

# Clean up the extracted folder
print(f"Cleaning up extracted folder: {data_base_path}")
if os.path.exists(data_base_path):
    shutil.rmtree(data_base_path)
    print("Cleanup successful.")
else:
    print("Folder not found, skipping cleanup.")

Loading 2018 data...
2018 data loaded: 216726 observations
Starting variable transformations...
Basic salary variables created.
String to numeric conversions completed.
Loading 2022 data...
2022 data loaded: 216726 observations
Starting variable transformations...
Basic salary variables created.
String to numeric conversions completed.
Loading 2010 data...
2010 data loaded: 216769 observations
Combined dataset after 2010: 673985 observations
Creating salary and time variables...
Salary and hours calculations completed.


c:\Users\Usuario\anaconda3\envs\tftimeseriesII\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\Usuario\anaconda3\envs\tftimeseriesII\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Creating dummy variables...
Age and sector dummies created.
Data processing completed successfully!
Final dataset shape: (883421, 123)
Years in dataset: [2010, 2014, 2018, 2022]
Cleaning up extracted folder: ..\data\EES10_14_18
Cleanup successful.


In [10]:
# Save the processed DataFrame to a Parquet file

# FIX: Convert all 'object' and 'category' dtype columns to 'string' for Parquet compatibility.
# This prevents errors with pyarrow when saving the file.
print("Fixing data types for Parquet compatibility...")
cols_to_convert = df.select_dtypes(include=['object', 'category']).columns

if len(cols_to_convert) > 0:
    print(f"Converting {len(cols_to_convert)} columns to string type: {list(cols_to_convert)}")
    for col in cols_to_convert:
        df[col] = df[col].astype(str)

# Save the file
output_path = os.path.join('..', 'data', 'EES_procesado.parquet')
print(f"Saving processed data to {output_path}...")
df.to_parquet(output_path)
print("Data saved successfully.")

Fixing data types for Parquet compatibility...
Converting 14 columns to string type: ['ordenccc', 'ordentra', 'region', 'secc', 'estrato2', 'cno2', 'responsa', 'cestudio', 'siespm1', 'siespm2', 'siespa1', 'siespa2', 'siespa3', 'siespa4']
Saving processed data to ..\data\EES_procesado.parquet...
Data saved successfully.
